In [ ]:
import os
from huggingface_hub import InferenceClient, AsyncInferenceClient, ChatCompletionInputTool
from dotenv import load_dotenv

load_dotenv()

client = InferenceClient(
    provider="openai",
    base_url="http://localhost:4000/v1",
    api_key=os.environ["LITELLM_API_KEY"],
)
async_client = AsyncInferenceClient(
    provider="openai",
    base_url="http://localhost:4000/v1",
    api_key=os.environ["LITELLM_API_KEY"],
)
tools = [
    ChatCompletionInputTool(
        type="function",
        function={
            "name": "sync_data",
            "description": "Synchronize database of lifelogs.",
            "parameters": {
                "type": "object",
                "properties": {
                    "sendSlackMessage": {
                        "type": "boolean",
                        "description": "If true, sends a Slack message."
                    }
                },
                # "required": ["sendSlackMessage"],
            }
        }
    )
]

In [6]:
completion = client.chat.completions.create(
    model="deepseek-v3",
    messages=[
        {
            "role": "user",
            "content": "How many 'G's in 'huggingface'?"
        }
    ],
)

print(completion.choices[0].message)

ChatCompletionOutputMessage(role='assistant', content='Alright, let\'s tackle the problem: **How many \'G\'s are in the word \'huggingface\'?**\n\n### Understanding the Problem\nFirst, I need to understand what the question is asking. We\'re given the word "huggingface," and we need to count how many times the letter \'G\' (uppercase or lowercase) appears in it.\n\n### Breaking Down the Word\nLet\'s write down the word and look at each letter one by one.\n\nThe word is: h u g g i n g f a c e\n\nNow, let\'s list out each letter with its position to keep track:\n\n1. h\n2. u\n3. g\n4. g\n5. i\n6. n\n7. g\n8. f\n9. a\n10. c\n11. e\n\n### Identifying the \'G\'s\nNow, let\'s go through each letter and see if it\'s a \'g\':\n\n1. h - not a g\n2. u - not a g\n3. g - yes, that\'s one\n4. g - yes, that\'s two\n5. i - not a g\n6. n - not a g\n7. g - yes, that\'s three\n8. f - not a g\n9. a - not a g\n10. c - not a g\n11. e - not a g\n\n### Counting the \'G\'s\nFrom the above, we\'ve identified t

In [23]:
import json


stream = client.chat.completions.create(
    model="nebius/deepseek-ai/DeepSeek-V3-0324-fast",
    messages=[
        {
            "role": "user",
            "content": "Run a sync with a slack notification"
        }
    ],
    tools=tools,
    tool_choice="auto",  # LLM decides if it needs a tool
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.tool_calls:
        for tool_call in chunk.choices[0].delta.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments or "{}")
            # Print the tool call name and arguments if available
            print(f"Tool call: {function_name}")
            if function_args:
                print(f"Args: {function_args}")
    elif chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)

Tool call: sync_data
Args: {'sendSlackMessage': True}
